In [1]:
import random
import numpy as np
from scipy.stats import multivariate_normal, norm
from scipy.optimize import minimize
import pandas as pd

In [2]:
###### FLEMING'S FUNCTION MODIFIED
def compute_meta_conf_serialdependence(xp, a, sigma_act, sigma_conf, rho):
    
    dhat = np.array([-1, 1])
    mu_x_xp_dhat = np.zeros((2, len(xp)))
    var_x_xp_dhat = np.zeros(len(xp))
    rho_vec = np.full(len(xp), rho)
    sigA_vec = np.full(len(xp), sigma_act)
    sigP_vec = np.full(len(xp), sigma_conf)
    
    Tol = 10e-4

    for dhati in range(2):
        dhat_vec = np.full(len(xp), dhat[dhati])
        
        mu_x_xp_dhat[dhati, :] = dhat_vec + (sigA_vec / sigP_vec) * rho_vec * (xp - dhat_vec)
        var_x_xp_dhat = (1 - rho_vec**2) * sigA_vec**2
        
        if a == 1:
            p_a_dhat_xp = 1 - norm.cdf(0, mu_x_xp_dhat[dhati, :], np.sqrt(var_x_xp_dhat))
        else:
            p_a_dhat_xp = norm.cdf(0, mu_x_xp_dhat[dhati, :], np.sqrt(var_x_xp_dhat))
        
        lik_d = norm.pdf(xp, dhat_vec, sigP_vec)
        
        if dhati == 0:
            p_a_dhat_xp_full = p_a_dhat_xp
            lik_d_full = lik_d
        else:
            p_a_dhat_xp_full = np.vstack((p_a_dhat_xp_full, p_a_dhat_xp))
            lik_d_full = np.vstack((lik_d_full, lik_d))
    
    # manage probability
    p_a_dhat_xp_full = np.clip(p_a_dhat_xp_full, Tol, None)
    lik_d_full = np.clip(lik_d_full, Tol, None)
    
    lik_d_full = lik_d_full / np.sum(lik_d_full, axis=0, keepdims=True)
    p_dhat_xp_a = p_a_dhat_xp_full * lik_d_full
    p_dhat_xp_a = p_dhat_xp_a / np.sum(p_dhat_xp_a, axis=0, keepdims=True)
    
    # Conf = p(a=d)
    if a == 1:
        conf = p_dhat_xp_a[1, :]
    else:
        conf = p_dhat_xp_a[0, :]
    
    return conf

In [3]:
def model_run(d, a, rho, sigmaConf, theta):
    sigmaAct = 1
    bigSigma = np.array([[sigmaAct**2, rho * sigmaAct * sigmaConf], [rho * sigmaAct * sigmaConf, sigmaConf**2]])

    N = len(d)
    xa = np.empty(N)
    xp = np.empty(N)
    secondOrder_mean_cor = np.empty(N)

    for i in range(N):
        current_theta = theta
        r = multivariate_normal.rvs(mean=[d[i] * current_theta, d[i] * current_theta], cov=bigSigma)
        
        xa[i] = r[0]
        xp[i] = r[1]
        
        flip_a = a[i]

        secondOrder_mean_cor[i] = compute_meta_conf_serialdependence(np.array([xp[i]]), flip_a, sigmaAct, sigmaConf, rho)[0]

    secondOrder_mean_cor_adj = secondOrder_mean_cor[1:]

    return secondOrder_mean_cor_adj

# Cargar datos y correr la optimización de los parámetros
df = pd.read_csv('data_Mazancieux_2018.csv')
#suj = [1,2,3]
#df_vp = df[(df['Task'] == 'VP') & (df['Subj_idx'].isin(suj))].copy()
df_vp = df[df['Task'] == 'EM'].copy()
df_vp.loc[:, 'Stimulus_transformed'] = df_vp['Stimulus'].replace({1: -1, 2: 1})
df_vp.loc[:, 'Response_transformed'] = df_vp['Response'].replace({1: 0, 2: 1})
df_vp['Confidence_0to1'] = df_vp['Confidence'] / 10

n_participants = len(df_vp['Subj_idx'].unique())

def negative_log_likelihood(params):
    rho = params[0]
    sigmaConf = params[1]
    theta = params[2]

    all_secondOrder_mean_cor = []

    for n_p in range(1, n_participants + 1): 
        df_vp_participant = df_vp[df_vp['Subj_idx'] == n_p]
        d = df_vp_participant['Stimulus_transformed'].values
        a = df_vp_participant['Response_transformed'].values
        
        secondOrder_mean_cor = model_run(d, a, rho, sigmaConf, theta)

        all_secondOrder_mean_cor.extend(secondOrder_mean_cor)

    all_secondOrder_mean_cor = np.array(all_secondOrder_mean_cor)

    grouped = df_vp.groupby('Subj_idx')
    df_vp_cleaned = pd.concat([group.iloc[1:] for _, group in grouped]).reset_index(drop=True)
    conf_data = df_vp_cleaned['Confidence_0to1'].values
   
    log_likelihood = np.sum(norm.logpdf(conf_data, all_secondOrder_mean_cor))

    print(f"Parameters: rho={rho}, sigmaConf={sigmaConf}, theta={theta}, Log-Likelihood: {-log_likelihood}")  # Añadí esta línea para imprimir los parámetros y el log-likelihood
   
    
    return -log_likelihood

# Inicializar valores iniciales para los parámetros
initial_guess = [0.5, 1.0, 0.5]
bounds = [(0.0001, 0.9999), (0.1, 2), (0.1, 0.9)]

result = minimize(negative_log_likelihood, initial_guess, bounds=bounds, method='Powell') # 'L-BFGS-B'
optimal_params = result.x
optimal_rho = optimal_params[0]
optimal_sigmaConf = optimal_params[1]
optimal_theta = optimal_params[2]

print("Optimal rho:", optimal_rho)
print("Optimal sigmaConf:", optimal_sigmaConf)
print("Optimal theta:", optimal_theta)

Parameters: rho=0.5, sigmaConf=1.0, theta=0.5, Log-Likelihood: 7097.1935500162535
Parameters: rho=0.3819896180478551, sigmaConf=1.0, theta=0.5, Log-Likelihood: 7095.176251330703
Parameters: rho=0.6180103819521449, sigmaConf=1.0, theta=0.5, Log-Likelihood: 7085.7993921081215
Parameters: rho=0.7638792360957102, sigmaConf=1.0, theta=0.5, Log-Likelihood: 7093.487584255459
Parameters: rho=0.58206876081791, sigmaConf=1.0, theta=0.5, Log-Likelihood: 7098.270509301377
Parameters: rho=0.618043717035855, sigmaConf=1.0, theta=0.5, Log-Likelihood: 7081.146279889996
Parameters: rho=0.6737479285497365, sigmaConf=1.0, theta=0.5, Log-Likelihood: 7074.5767698019545
Parameters: rho=0.6459193889037202, sigmaConf=1.0, theta=0.5, Log-Likelihood: 7087.823136175386
Parameters: rho=0.7081750245818286, sigmaConf=1.0, theta=0.5, Log-Likelihood: 7088.655170387638
Parameters: rho=0.6868979091000391, sigmaConf=1.0, theta=0.5, Log-Likelihood: 7093.785374721276
Parameters: rho=0.6631183722622322, sigmaConf=1.0, thet

In [4]:
'''
Optimal rho: 0.7668371318658034
Optimal sigmaConf: 1.1859845886834475
Optimal theta: 0.2862428034906346
'''

'\nOptimal rho: 0.7668371318658034\nOptimal sigmaConf: 1.1859845886834475\nOptimal theta: 0.2862428034906346\n'

In [5]:
import csv

# Guardar en un archivo CSV
with open('EMtask_optimal_parameters_model_4_withoutSerialDependence.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Parameter', 'Value'])
    writer.writerow(['rho', optimal_rho])
    writer.writerow(['sigmaConf', optimal_sigmaConf])
    writer.writerow(['theta', optimal_theta])